<h1> Problem 75 - Singular Integer Right Triangles </h1> 

It turns out that 12 cm is the smallest length of wire that can be bent to form an integer sided right angle triangle in exactly one way, but there are many more examples.
\begin{align*}
    (3,4,5) & = 12 \\
    (6,8,10) & = 24 \\
    (5,12,13) & = 30 \\
    (9,12,15) & = 36 \\
    (8,15,17) & = 40 \\
    (12,16,20) & = 48 
\end{align*}
In contrast, some lengths of wire, like 20 cm, can't be bent to form an integer sided right angle triangle, and other lenths allow more than one solution to be found; for example, using 120 cm it is possible to form exactly three different integer sided right angle triangles. 
\begin{equation*}
    (30,40,50) = (20,48,52) = (24,45,51) = 120
\end{equation*}
Given that $L$ is the length of the wire, for how many values of $L \leq 1500000$ can exactly one integer sided right angle triangle be formed. 

---


Properties to exploit:
1. A property of the sum of pythagorean triplets is that it will always be even. This automatically eliminates half of all possible values. 
2. From one primitive pythagorean triplet, I can find all sums that are a multiple of it. If a number is a multiple of more than one sum, then it will have more than one solution. For example, $120 = 30 \times 4 = 12 \times 10 = 40 \times 3$. Therefore, from the first $x$ number of primitives, I can eliminate most sums.
3. I can terminate when every even number less than $1500000$ can be expressed as a multiple of my primitives. I don't think I need to go all the way to $1500000$. 

The real question is, is this equivalent to just finding all the primitives? 

[Euclid's formula to generate Pythagorean Triplets](https://en.wikipedia.org/wiki/Pythagorean_triple#Generating_a_triple), states that, for an aribitrary pair of integers, $m$ and $n$, with $m>n>0$, then 
\begin{equation*}
    x = m^2 - n^2 \qquad y = 2mn \qquad z = m^2 + n^2
\end{equation*}
where $(x, y, z)$ is a pythagorean triplet that fulfills $x^2 + y^2 = z^2$, $m$ and $n$ are coprime (i.e. $gcd(m,n) = 1$) integers, and $m$ and $n$ are of opposite parity (i.e. if $m$ is even, then $n$ is odd, and vice versa). 

I think if I solve for all the pythogorean triplets up to, 
\begin{equation*}
    w = 2m(m+n)
\end{equation*}
where $w$ is the perimeter, it should be fine. This also implies that the perimeter will always be even. An alternative proof would be to consider $x^2 \equiv x \pmod{2}$. This is equivalent to saying if $x^2$ is even then $x$ will be even as well, and if $x^2$ is odd, so will $x$. Therefore, 
\begin{equation*}
    x + y + z \equiv x^2 + y^2 + z^2 \equiv z^2 + z^2 \equiv 2z^2 \pmod{2}
\end{equation*}
I wish I could claim credit for this beautifully elegant proof, but [this post](https://math.stackexchange.com/a/2162893) is what enlightened me to the beauty of it. 

[All pairs of coprime intergers](https://en.wikipedia.org/wiki/Coprime_integers#Generating_all_coprime_pairs), $m$ and $n$ with $m>n$ are known to be given by, 
\begin{align}
    (2m - n ,m) & \\
    (2m + n ,m) & \\
    (m + 2n, n)
\end{align}
where each equation represents one branch of a [ternary tree](https://en.wikipedia.org/wiki/Ternary_tree). This scheme is known to be exhaustive and non-redundant with not invalid memebers [cite](https://en.wikipedia.org/wiki/Coprime_integers#Generating_all_coprime_pairs). Since pythagorean triplets require either odd-even pairs or even-odd pairs, then the starting point for each branch will be $(2,1)$.

It can be guaranteed that the $m$ and $n$ values will be larger than the previous iteration, one can safely assume that $x$, $y$ and $z$ will be larger and hence their sum will be larger. 

Actually, if I just need to find the number of primitives, then I don't ever need to find the triplets. I can just find the number of terms that fulfill the coprime conditions. *It is not equivalent, the second example in the question is an example of where it is a non-primitve but unique triplet*. Here, unique is defined as the sum of $x$, $y$, $z$ not being repeated. 

In [2]:
use std::collections::HashMap;
use std::error::Error;

#[derive(Hash, Eq, PartialEq, Debug, Default, Clone)]
struct PythagoreanTriplet {
    branch_1: (u32, u32),
    branch_2: (u32, u32),
    branch_3: (u32, u32),
    perimeter: Vec<u32>,
    which_branches_alive: Vec<bool>,
    are_all_branches_dead: bool,
    number_of_alive_branches: u8,
}

impl PythagoreanTriplet {
    
    fn first_triplet() -> PythagoreanTriplet {
        PythagoreanTriplet {
            branch_1: (0,0),
            branch_2: (0,0),
            branch_3: (0,0),
            perimeter: vec![12, 0, 0],
            which_branches_alive: vec![true, false, false],
            are_all_branches_dead: false,
            number_of_alive_branches: 1,
        }
    }
    
    fn new(m_0: &u32, n_0: &u32, branch_status: &Vec<bool>, max_value: u32) -> PythagoreanTriplet{
        
        let mut output = PythagoreanTriplet::default();
        
        output.perimeter = vec![0;3];
        
        for i in 0..3 {
            if branch_status[i] {
                output.compute_for_branch(i + 1, *m_0, *n_0);
            } else {
                output.set_dead_branch_values(i + 1);
            }
        }
        
        output.which_branches_alive = output.perimeter
                                                .iter()
                                                .map(|p| 
                                                    if p > &max_value || p == &0
                                                        {false} 
                                                    else {true})
                                                .collect();
        output.number_of_alive_branches = output.which_branches_alive.iter().filter(|&x| x == &true).count() as u8;
//         println!("{:?} {:?}", output.which_branches_alive, output.number_of_alive_branches);
        
        if output.number_of_alive_branches == 0 {
            output.are_all_branches_dead = true;
        } else {
            output.are_all_branches_dead = false;
        }
        output
    }
    
    fn get_number_alive_branches(&self) -> u8 {
        self.number_of_alive_branches
    }
    
    fn get_are_all_branches_dead(&self) -> bool {
        self.are_all_branches_dead
    }
    
    fn get_branch_status(&self) -> &Vec<bool> {
        &self.which_branches_alive
    }
    
    fn set_dead_branch_values(& mut self, branch_num: usize) 
    {
            
        match branch_num {
            1 => self.branch_1 = (0,0),
            2 => self.branch_2 = (0,0),
            3 => self.branch_3 = (0,0),
            _ => panic!(),
        };
        
        self.perimeter[branch_num -1 ] = 0;
            
    }
    
    fn compute_for_branch(&mut self, branch_num: usize, m: u32, n: u32)
    {
        match branch_num {
            1 => {
                let new_val = PythagoreanTriplet::next_branch_1_pair(m, n);
                self.perimeter[branch_num - 1] = PythagoreanTriplet::compute_perimeter(new_val.0, new_val.1);
                self.branch_1 = new_val;
            },
            2 => {
                let new_val = PythagoreanTriplet::next_branch_2_pair(m, n);
                self.perimeter[branch_num - 1] = PythagoreanTriplet::compute_perimeter(new_val.0, new_val.1);
                self.branch_2 = new_val;
            },
            3 => {
                let new_val = PythagoreanTriplet::next_branch_3_pair(m, n);
                self.perimeter[branch_num - 1] = PythagoreanTriplet::compute_perimeter(new_val.0, new_val.1);
                self.branch_3 = new_val;
            },
            _ => panic!(),
        };
    }
    
    fn compute_perimeter(m: u32, n: u32) -> u32 {
        2*m*(m+n)
    }
    
    fn is_max_perimeter_exceeded(max_perimeter: u32, m_and_n: (u32, u32)) -> bool {
        let perimeter = PythagoreanTriplet::compute_perimeter(m_and_n.0, m_and_n.1);
//         println!("m = {} n = {} perimeter = {}", m_and_n.0, m_and_n.1, perimeter);
        if perimeter > max_perimeter {
            true
        } else {
            false
        } 
    }
    
    
    fn next_branch_1_pair(m: u32, n: u32) -> (u32, u32) {
        (2*m -n , m)
    }

    fn next_branch_2_pair(m: u32, n:u32) -> (u32, u32) {
        (2*m + n, m)
    }

    fn next_branch_3_pair(m: u32, n:u32) -> (u32, u32) {
        (m + 2*n, n)
    }
    
}

In [3]:
struct PythagoreanTripletIterator<'a> {
    triplet: &'a PythagoreanTriplet, 
    index: usize,
}


#[derive(Debug )]
struct BranchInfo {
    m: u32,
    n: u32, 
    alive: bool,
    perimeter: u32,
}

impl BranchInfo {
    fn new(m: u32, n: u32, alive:bool, perimeter: u32) -> BranchInfo {
        BranchInfo{m, n, alive, perimeter}
    }
}


impl<'a> Iterator for PythagoreanTripletIterator<'a> {
    type Item = BranchInfo;
    
    fn next(&mut self) -> Option<BranchInfo> {
        let branch_value = match self.index {
            0 => self.triplet.branch_1, 
            1 => self.triplet.branch_2,
            2 => self.triplet.branch_3,
            _ => return None, 
        };
        let branch_status = self.triplet.get_branch_status()[self.index];
        let perimeter = self.triplet.perimeter[self.index];
        self.index += 1;
        
        Some(BranchInfo::new(branch_value.0, branch_value.1, branch_status, perimeter))
    }
}

impl PythagoreanTriplet {
    fn iter(&self) -> PythagoreanTripletIterator {
        PythagoreanTripletIterator{
            triplet: self,
            index: 0
        }
    }
}


In [4]:
fn create_key_name(generation_number: u32, child_number: u32) -> String {
    String::from(vec![generation_number.to_string(), child_number.to_string()].join("_"))
}

In [5]:
use std::collections::HashMap;

fn assemble_primitives(max_value: u32) -> HashMap<String, PythagoreanTriplet> {
    let mut all_triplets = HashMap::new();
    
    let mut number_of_primitives: u32 = 1;
    
    let mut which_generation = 0;
    let mut which_child_in_generation = 1;
    
    let initial_m = 2;
    let initial_n = 1;
    let initial_status = vec![true;3];
    
    let mut key_name = create_key_name(which_generation, which_child_in_generation);
    
    all_triplets.insert(key_name, PythagoreanTriplet::new(&initial_m, &initial_n, &initial_status, max_value));
    
    loop {
        let key_subset =  format!("{}_", which_generation);
        
        let children_in_this_generation: HashMap< String,  PythagoreanTriplet> = all_triplets
                                                                                    .clone()
                                                                                    .into_iter()
                                                                                    .filter(|(k, _)| k.contains(& key_subset))
                                                                                    .collect::<HashMap<String, PythagoreanTriplet>>();
        
        let number_of_live_children = children_in_this_generation.len();
        
        if number_of_live_children == 0 {
            break
        }
        
        // Create the next generation 
        which_generation += 1;
        which_child_in_generation = 0;
        
        for (_, parent_triplet) in children_in_this_generation.iter().filter(|(_, v)| !v.get_are_all_branches_dead()){
            
            for branch in parent_triplet.iter().filter(|x| x.alive) {
//                 println!("{:?}", branch);
                
                key_name = create_key_name(which_generation, which_child_in_generation);
                all_triplets.insert(key_name, PythagoreanTriplet::new(&branch.m, &branch.n, parent_triplet.get_branch_status(), max_value));
                which_child_in_generation += 1;
                number_of_primitives += 1;
                
            }
                
        }
        
    }
    all_triplets.insert(String::from("0"), PythagoreanTriplet::first_triplet());
    
    return all_triplets
}


In [6]:
fn extract_half_perimeters(primitives: &HashMap<String, PythagoreanTriplet>) -> Vec<u32> {
    primitives.iter().flat_map(|(_, v)| v.iter()
                                    .filter(|branch| branch.alive)
                                    .map(|branch| branch.perimeter/2)
                                    .collect::<Vec<u32>>())
                     .collect::<Vec<u32>>()
    
}

In [7]:
// I can either use a lot of space and do something similar to the seive of erosthenes
// Or I can use more computational time and test each number to see if it is a multiple

fn unique_perimeters_space(half_perimeters: Vec<u32>, max_perimeter: u32) -> u32 {
    let half_perimeter = max_perimeter/2;
    let mut perimeters : Vec<u8> = vec![0 as u8; (half_perimeter + 1) as usize];
    
    for item in half_perimeters {
        let num_multiples = half_perimeter/item;
        
        (1..num_multiples+1).into_iter()
                            .for_each(|x| perimeters[(x*item) as usize] += 1);
    }
//     println!("{:?}", perimeters);
    perimeters.iter().filter(|x| x == &&1).count() as u32
}

In [8]:
// let n = 100;
let n = 1500000;
let primitives = assemble_primitives(n);
let all_primitive_perimeters = extract_half_perimeters(&primitives);
let num_unique = unique_perimeters_space(all_primitive_perimeters, n);
println!("There are {} unique perimiters for L <= 1 500 000", num_unique);

There are 161667 unique perimiters for L <= 1 500 000


<h2> How does this method work? </h2> 

**Step 1:** Find all the primitive Pythagorean Triplets with a total perimeter of less than $L$. 

**Step 2:** Get the perimeters of all of the primitive triplets. Since the perimeters will always be even, divide all by 2 to reduce the space complexity of the next step. 

**Step 3:** Using the known primitive triplets, find all the multiple of it below $L$. Increment the counter associated with that value by 1. This method is very inefficient in terms of space/memory requirements as it requires a vector of size $L/2$. 

**Step 4:** Count the number of entries in the vector that have a counter value of $1$. 

**General comments of this method:**
1. It is slow. I think computationally, it can be made more efficient by a) removing the clone of the hashmap that contains all triplets (this was put in place as the hashmap was borrowed mutably and immutably in the same scope) b) using a ternary tree instead of the structs? I did this was I wanted to see if I can find a way around it c) matrix vector multiplication could also result in improvements but I wanted to do try to do this without an external crate. However, this seems to be causing quite a bit of overhead
2. The counting of the number of unique perimeters takes up *a lot* of space. I've used a similar concept to the seive of erosthenes to "count up" numbers that have multiples. But, it will *always* require a vector of size $L/2$. Perhaps, there is an alternative that has a lower space complexity but I got lazy at the end. 


**How is step 1 found?**
1. Euclid's formula allows the terms of the triplet ($x$, $y$, $z$) to two variables $m$ and $n$. 
2. A condition of $m$ and $n$ is that they must be coprime and be of opposite parity. Given that all coprime integers lie on three branches and selecting the starting value to be $(2,1)$, this allows for all $m$ and $n$ values to be found. Therefore, their $gcd$ never need to be found or tested if they are are of opposite parity.
3. Using this method also guarantees that $m$ and $n$ will always increase. Hence, once the upper bound is exceeded, then the algorithm can safely terminate.
4. Once the maximum perimeter is exceeded, a branch is deemed to have "died" such that it is unable to propagate.

**What property does step 3 use?**

From [Euclid's formula](https://en.wikipedia.org/wiki/Pythagorean_triple#Generating_a_triple), it is known that all non-primitve triplets can be found using, 
\begin{equation*}
    a = k(m^2 - n^2) \quad b = k(2mn) \quad c = k(m^2 + n^2)
\end{equation*}
where $m$, $n$, $k$ are positive integers with $m>n$ and the aforementioned criterion for $m$ and $n$ hold true.
